<a href="https://colab.research.google.com/github/mmdebrahimi/Deep-Reinforcement-Learning/blob/main/Grid_search_Optimization_Stock_market.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###GET DATA: 
- In this section we first make sure alpaca trading api is installed and then get the data through the API key.
- We then have to add the indicators that we wish to use in our grid search, adding them to our DataFrame
-  the price is set as the target and X_train, y_train, and X_test and y_test are defined
- The parameter space for our grid search and our methodology(random forest, SVC, etc.) is defined.
- grid search is implemented and best scores and best parameters are returned to optimize our indicators

In [2]:
!pip install alpaca_trade_api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 112 kB 37.8 MB/s 
     |████████████████████████████████| 1.1 MB 50.3 MB/s 
     |████████████████████████████████| 596 kB 38.5 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 144 kB 51.7 MB/s 
     |████████████████████████████████| 271 kB 43.8 MB/s 
     |████████████████████████████████| 94 kB 2.7 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import REST, TimeFrame

BASE_URL = "https://paper-api.alpaca.markets"
ALPACA_API_KEY = "PKQ13N77DB9EQFHCS8RT"
ALPACA_SECRET_KEY = "XDLrABswxDeDp79nGkp1KZIqak1ZpPZAsdNZkVVY"

# Instantiate REST API Connection
api = tradeapi.REST(key_id=ALPACA_API_KEY, secret_key=ALPACA_SECRET_KEY, 
                    base_url=BASE_URL, api_version='v2')

spy = api.get_bars("SPY", TimeFrame.Day, "2015-12-01", "2022-05-01", adjustment='raw').df

In [4]:
display(spy)


,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2015-12-01 05:00:00+00:00,209.42,210.8200,209.1100,210.68,97858418,337780,209.921410
2015-12-02 05:00:00+00:00,210.60,211.0000,208.2300,208.54,108069059,367013,209.563055
2015-12-03 05:00:00+00:00,208.90,209.1500,204.7511,205.58,166224154,546768,206.878936
2015-12-04 05:00:00+00:00,206.10,209.9700,205.9300,209.66,192878747,556731,208.178631
2015-12-07 05:00:00+00:00,209.20,209.7295,207.2000,208.27,102027111,374705,208.276128
...,...,...,...,...,...,...,...
2022-04-25 04:00:00+00:00,423.70,428.6900,418.8400,428.51,119103454,1139830,423.952513
2022-04-26 04:00:00+00:00,425.92,426.0400,416.0700,416.10,101074178,1001090,420.127020
2022-04-27 04:00:00+00:00,417.38,422.9200,415.0100,417.20,120771109,1005154,418.848698


###GRID SEARCH:
    - An instance of pipeline is created using make_pipeline method from sklearn.pipeline. The instance of pipeline is passed to GridSearchCV via estimator
    - A JSON array of parameter grid is created for passing the same to GridSearchCV via param_grid
    - Cross-validation generator is passed to GridSearchCV. In the example given in this post, the default such as StratifiedKFold is used by passing cv = 10
    - Another parameter, refit = True, is used which refit the the best estimator to the whole training set automatically.
    - The scoring parameter is set to ‘accuracy’ to calculate the accuracy score.
    - Method, fit, is invoked on the instance of GridSearchCV with training data (X_train) and related label (y_train).
    - Once the GridSearchCV estimator is fit, the following attributes are used to get vital information:
        1 - best_score_: Gives the score of the best model which can be created using most optimal combination of hyper parameters
        2 - best_params_: Gives the most optimal hyper parameters which can be used to get the best model
        3- best_estimator_: Gives the best model built using the most optimal hyperparameters

In [5]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [6]:
#define pipeline parameters - set to random forest classifier
pipelineRFC = make_pipeline(StandardScaler(), RandomForestClassifier(criterion='gini', random_state=1))

# Create the parameter grid
#
param_grid_rfc = [{
    'randomforestclassifier__max_depth':[2, 3, 4],
    'randomforestclassifier__max_features':[2, 3, 4, 5, 6]
}]
#
# Create an instance of GridSearch Cross-validation estimator
# I wonder if this approach will work as crossvalidation and stock market don't make sense together --> needs more research
gsRFC = GridSearchCV(estimator=pipelineRFC,
                     param_grid = param_grid_rfc,
                     scoring='accuracy',
                     cv=10,
                     refit=True,
                     n_jobs=1)
#
# Train the RandomForestClassifier
#
# gsRFC = gsRFC.fit(X_train, y_train) --> activate once you have defined X_train and y_train
#
# Print the training score of the best model
#
# print(gsRFC.best_score_)
# #
# # Print the model parameters of the best model
# #
# print(gsRFC.best_params_)
# #
# # Print the test score of the best model
# #
# clfRFC = gsRFC.best_estimator_
# print('Test accuracy: %.3f' % clfRFC.score(X_test, y_test))

In [ ]:
#SVC based Grid Search
# pipelineSVC = make_pipeline(StandardScaler(), SVC(random_state=1))
# #
# # Create the parameter grid
# #
# param_grid_svc = [{
#                     'svc__C': [0.001, 0.01, 0.05, 0.1, 0.5, 1.0, 10.0],
#                     'svc__kernel': ['linear']
#                   },
#                  {
#                     'svc__C': [0.001, 0.01, 0.05, 0.1, 0.5, 1.0, 10.0],
#                     'svc__gamma': [0.001, 0.01, 0.05, 0.1, 0.5, 1.0, 10.0],
#                     'svc__kernel': ['rbf']
#                  }]
# #
# # Create an instance of GridSearch Cross-validation estimator
# #
# gsSVC = GridSearchCV(estimator=pipelineSVC,
#                      param_grid = param_grid_svc,
#                      scoring='accuracy',
#                      cv=10,
#                      refit=True,
#                      n_jobs=1)
# #
# # Train the SVM classifier
# #
# gsSVC.fit(X_train, y_train)
# #
# # Print the training score of the best model
# #
# print(gsSVC.best_score_)
# #
# # Print the model parameters of the best model
# #
# print(gsSVC.best_params_)
# #
# # Print the model score on the test data using GridSearchCV score method
# #
# print('Test accuracy: %.3f' % gsSVC.score(X_test, y_test))
# #
# # Print the model score on the test data using Best estimator instance
# #
# clfSVC = gsSVC.best_estimator_
# print('Test accuracy: %.3f' % clfSVC.score(X_test, y_test))

In [ ]:
# #Logistic Regression based grid search
# pipelineLR = make_pipeline(StandardScaler(), LogisticRegression(random_state=1, penalty='l2', solver='lbfgs'))
# #
# # Create the parameter grid
# #
# param_grid_lr = [{
#     'logisticregression__C': [0.001, 0.01, 0.05, 0.1, 0.5, 1.0, 10.0],
# }]
# #
# # Create an instance of GridSearch Cross-validation estimator
# #
# gsLR = GridSearchCV(estimator=pipelineLR,
#                      param_grid = param_grid_lr,
#                      scoring='accuracy',
#                      cv=10,
#                      refit=True,
#                      n_jobs=1)
# #
# # Train the LogisticRegression Classifier
# #
# gsLR = gsLR.fit(X_train, y_train)
# #
# # Print the training score of the best model
# #
# print(gsLR.best_score_)
# #
# # Print the model parameters of the best model
# #
# print(gsLR.best_params_)
# #
# # Print the test score of the best model
# #
# clfLR = gsLR.best_estimator_
# print('Test accuracy: %.3f' % clfLR.score(X_test, y_test))